In [0]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np
import scipy.io

In [3]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [0]:
from scipy.io import loadmat

train = loadmat('gdrive/My Drive/train_32x32.mat')
test = loadmat('gdrive/My Drive/train_32x32.mat')


X_train = train['X']
y_train = train['y']
X_test = test['X']
y_test = test['y']


In [23]:
print("Shape of X_train is:", X_train.shape)
print("Shape of y_train is:", y_train.shape)
print("Shape of X_test is:", X_test.shape)
print("Shape of y_test is:", y_test.shape)

Shape of X_train is: (32, 32, 3, 73257)
Shape of y_train is: (73257, 1)
Shape of X_test is: (32, 32, 3, 73257)
Shape of y_test is: (73257, 1)


In [24]:
X_train = X_train.transpose((3,0,1,2))
X_test = X_test.transpose((3,0,1,2))

print("Shape of X_train is now:", X_train.shape)
print("Shape of X_test is now:", X_test.shape)

y_train = y_train[:,0]
y_test = y_test[:,0]

print("Shape of y_train is now:", y_train.shape)
print("Shape of y_test is now:", y_test.shape)

y_train[y_train==10] = 0
y_test[y_test==10] = 0

print("labels of y_train are", np.unique(y_train[:]))
print("labels of y_test are", np.unique(y_test[:]))

Shape of X_train is now: (73257, 32, 32, 3)
Shape of X_test is now: (73257, 32, 32, 3)
Shape of y_train is now: (73257,)
Shape of y_test is now: (73257,)
labels of y_train are [0 1 2 3 4 5 6 7 8 9]
labels of y_test are [0 1 2 3 4 5 6 7 8 9]


In [29]:
X_train = X_train / 255
X_test = X_test / 255
print(y_train)

[1 9 2 ... 1 6 9]


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout


model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(32, 32, 3)),
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.3),
    Conv2D(64, (3,3), activation='relu'),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.3),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='softmax')
])

# get a summary of our built model
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 10, 10, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
__________

In [17]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train, y_train, epochs=40)
loss, acc = model.evaluate(X_test, y_test)
print("Model accuracy on test data is: {:6.3f}%".format(100 * acc))
model.save('my_model.h5')

Epoch 1/40
73257/73257 [==============================] - 22s 297us/sample - loss: 1.1614 - acc: 0.6030
Epoch 2/40
73257/73257 [==============================] - 15s 203us/sample - loss: 0.4672 - acc: 0.8575
Epoch 3/40
73257/73257 [==============================] - 15s 200us/sample - loss: 0.4007 - acc: 0.8791
Epoch 4/40
73257/73257 [==============================] - 15s 198us/sample - loss: 0.3639 - acc: 0.8909
Epoch 5/40
73257/73257 [==============================] - 16s 213us/sample - loss: 0.3393 - acc: 0.8978
Epoch 6/40
73257/73257 [==============================] - 15s 199us/sample - loss: 0.3172 - acc: 0.9046
Epoch 7/40
73257/73257 [==============================] - 15s 199us/sample - loss: 0.3009 - acc: 0.9091
Epoch 8/40
73257/73257 [==============================] - 14s 197us/sample - loss: 0.2895 - acc: 0.9124
Epoch 9/40
73257/73257 [==============================] - 15s 198us/sample - loss: 0.2845 - acc: 0.9136
Epoch 10/40
73257/73257 [==============================] - 15s 2

In [0]:
import tensorflow as tf 
new_model = tf.keras.models.load_model('my_model.h5')

In [37]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)
y_train = y_train[:,0]
y_test = y_test[:,0]
print(y_train.shape)
#num_classes = 10
#y_train = np_utils.to_categorical(y_train,num_classes)
#y_test = np_utils.to_categorical(y_test,num_classes)

(50000,)


In [40]:
new_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
new_model.fit(x_train, y_train, epochs=60)
loss, acc = new_model.evaluate(x_test, y_test)
print("Model accuracy on test data is: {:6.3f}%".format(100 * acc))

Epoch 1/60
50000/50000 [==============================] - 10s 205us/sample - loss: 0.5720 - acc: 0.8048
Epoch 2/60
50000/50000 [==============================] - 10s 195us/sample - loss: 0.5773 - acc: 0.8029
Epoch 3/60
50000/50000 [==============================] - 11s 218us/sample - loss: 0.5828 - acc: 0.8003
Epoch 4/60
50000/50000 [==============================] - 10s 194us/sample - loss: 0.5620 - acc: 0.8068
Epoch 5/60
50000/50000 [==============================] - 10s 195us/sample - loss: 0.5551 - acc: 0.8086
Epoch 6/60
50000/50000 [==============================] - 10s 195us/sample - loss: 0.5625 - acc: 0.8053
Epoch 7/60
50000/50000 [==============================] - 10s 194us/sample - loss: 0.5629 - acc: 0.8069
Epoch 8/60
50000/50000 [==============================] - 10s 194us/sample - loss: 0.5645 - acc: 0.8065
Epoch 9/60
50000/50000 [==============================] - 10s 194us/sample - loss: 0.5630 - acc: 0.8073
Epoch 10/60
50000/50000 [==============================] - 10s 1